In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import re
import json


caps = "([A-Z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + caps + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(caps + "[.]" + caps + "[.]" + caps + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(caps + "[.]" + caps + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + caps + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [2]:
def get_response_url(partial_url):
    full_url = 'https://vng.nl{}'.format(partial_url)
    r = requests.get(full_url)
    return r.url

def get_soup_from_partial_url(partial_url):
    full_url = 'https://vng.nl{}'.format(partial_url)
    r = requests.get(full_url)
    return BeautifulSoup(r.content, "html5lib")

def get_news_article_urls_from_page(soup):
    urls = []
    
    for a_tag in soup.find_all('a', href=True):

        if '/nieuws/' in a_tag['href']:
            urls.append(a_tag['href'])
            
    return urls

def next_page_soup(soup):    
    next_page_button = soup.find('li', {'class' : 'pager-next'})
    
    if next_page_button:
        return get_soup_from_partial_url(next_page_button.a['href'])
    else:
        return None

    
def read_news_content_from_url(url):
    page_soup = get_soup_from_partial_url(url)
    article = page_soup.find('div', {'about' : url})
    if article:
        content = article.find('div', {'class' : 'content'})
        sentences = split_into_sentences(content.get_text())
    else:
        sentences = []
    
    for sentence in sentences:
        if 'vng-dossier' in sentence.lower():
            sentences.remove(sentence)
    
    return ' '.join(sentences)

def read_label_from_url(url):
    splitted = url.split('/')
    return splitted[2]

In [3]:
urls = []

current_soup = get_soup_from_partial_url('/nieuws')

while current_soup:
    current_urls = get_news_article_urls_from_page(current_soup)
    
    for url in current_urls:
        urls.append(url)
    print(len(urls))
    
    current_soup = next_page_soup(current_soup)

14
24
34
44
54
64
74
84
94
104
114
124
134
144
154
164
174
184
194
204
214
224
234
244
254
264
274
284
294
304
314
324
334
344
354
364
374
384
394
404
414
424
434
444
454
464
474
484
494
504
514
524
534
544
554
564
574
584
594
604
614
624
634
644
654
664
674
684
694
704
714
724
734
744
754
764
774
784
794
804
814
824
834
844
854
864
874
884
894
904
914
924
934
944
954
964
974
984
994
1004
1014
1024
1034
1044
1054
1064
1074
1084
1094
1104
1114
1124
1134
1144
1154
1164
1174
1184
1194
1204
1214
1224
1234
1244
1254
1264
1274
1284
1294
1304
1314
1324
1334
1344
1354
1364
1374
1384
1394
1404
1414
1424
1434
1444
1454
1464
1474
1484
1494
1504
1514
1524
1534
1544
1554
1564
1574
1584
1594
1604
1614
1624
1634
1644
1654
1664
1674
1684
1694
1704
1714
1724
1734
1744
1754
1764
1774
1784
1794
1804
1814
1824
1834
1844
1854
1864
1874
1884
1894
1904
1914
1924
1934
1944
1954
1964
1974
1984
1994
2004
2014
2024
2034
2044
2054
2064
2074
2084
2094
2104
2114
2124
2134
2144
2154
2164
2174
2184
2194
2204
2214
222

In [4]:
result = []

for url in urls:
    print(url)
    
    content = read_news_content_from_url(url)
    response_url = get_response_url(url)

    main = response_url.split('/')[4]
    sub = response_url.split('/')[5]
    
    result.append({'main_topic' : main, 'sub_topic' : sub, 'content' : content})
    

with open('../data_resources/topics/vng_training.json', 'w') as outfile:
    json.dump(result, outfile)

/nieuws/18-02-09/monitor-jeugdregios-goede-start-met-ambities-zorglandschap
/nieuws/18-02-09/brede-oproep-voor-integrale-voorziening-voor-alle-peuters
/nieuws/18-02-09/vng-haalt-input-op-bij-leden-voor-klimaatakkoord
/nieuws/18-02-08/2018-het-jaar-van-de-nieuwe-privacyregels-van-de-avg
/nieuws/18-02-09/monitor-jeugdregios-goede-start-met-ambities-zorglandschap
/nieuws/18-02-09/blokhuis-met-gemeenten-in-gesprek-over-publieke-gezondheid
/nieuws/18-02-09/brede-oproep-voor-integrale-voorziening-voor-alle-peuters
/nieuws/18-02-09/voortgang-nationaal-preventieakkoord
/nieuws/18-02-09/vng-haalt-input-op-bij-leden-voor-klimaatakkoord
/nieuws/18-02-09/jep-helpt-jeugdprofessionals-bij-vragen-over-extremisme
/nieuws/18-02-09/discussie-comite-van-de-regios-over-europese-ambities
/nieuws/18-02-09/wnra-enquete-bezwaar-en-beroep
/nieuws/18-02-09/zicht-op-gemeentelijke-schuldhulpverlening-doe-de-quickscan
/nieuws/18-02-09/conferentie-connecting-territories
/nieuws/18-02-09/nijmegen-uitgeroepen-tot-gre

/nieuws/18-01-18/noodwet-nodig-om-meer-te-bouwen
/nieuws/18-01-18/corporatie-zorg-en-gemeente-werken-samen-in-weer-thuis
/nieuws/18-01-18/startup-europe-regions-network-van-start-in-boedapest
/nieuws/18-01-17/aanstellingswijze-volgt-uit-rol-en-positie-burgemeester
/nieuws/18-01-17/vernieuwing-beschermd-wonen-is-ver-in-midden-holland
/nieuws/18-01-17/ding-mee-naar-de-eu-sustainable-energy-awards
/nieuws/18-01-17/nieuwe-online-tool-beslisboom-verduurzaming-scholen
/nieuws/18-01-17/nieuwsbrief-professionalisering-en-kwaliteit-van-jeugdhulp
/nieuws/18-01-17/wetsvoorstel-knelpunten-jeugdwet-en-wmo-2015-naar-de-kamer
/nieuws/18-01-17/digitale-folder-screeningen-bij-pasgeborenen-beschikbaar
/nieuws/18-01-16/beperkte-stijging-woonlasten-grote-gemeenten
/nieuws/18-01-16/wie-wordt-de-meest-inspirerende-global-goals-gemeente
/nieuws/18-01-16/start-programma-duurzameinclusieve-stedelijke-ontwikkeling
/nieuws/18-01-16/investeren-in-pleegzorg-loont-gemeenten-werken-eraan
/nieuws/18-01-15/alles-in-ee

/nieuws/17-12-13/kinderopvang-helpt-arbeidsparticipatie-en-ontwikkeling-kind
/nieuws/17-12-13/oproep-aan-tk-geen-nieuwe-salderingsregeling-voor-2023
/nieuws/17-12-13/maximumtarief-rijbewijs-2018-bekend
/nieuws/17-12-13/schrijf-uw-stad-in-voor-nieuwe-europese-award
/nieuws/17-12-13/pleidooi-voor-een-overheid-die-zicht-houdt-op-digitalisering
/nieuws/17-12-13/laat-tractoren-niet-racen-in-de-bebouwde-kom
/nieuws/17-12-13/aantal-who-verzoeken-is-beperkt-kosten-ook
/nieuws/17-12-12/geef-gemeenten-de-regie-op-inburgering-vergunninghouders
/nieuws/17-12-12/richtlijn-vechtsportevenementen-als-toets-vergunningenbeleid
/nieuws/17-12-12/kamerbrief-over-de-kwaliteit-van-het-leerlingenvervoer
/nieuws/17-12-12/lees-het-magazine-over-de-jongvolwassenentop
/nieuws/17-12-12/begrotingsbehandeling-ezk-aandachtspunten-energietransitie
/nieuws/17-12-11/steun-ned-gemeenten-aan-st-maarten-saba-en-st-eustatius
/nieuws/17-12-11/decembercirculaire-gemeentefonds-2017-is-beschikbaar
/nieuws/17-12-11/handhaving-en

/nieuws/17-11-27/gezamenlijk-manifest-minder-zelfredzame-jongeren-getekend
/nieuws/17-11-27/inspiratiemagazine-gezondheid-statushouders
/nieuws/17-11-27/webtool-gegevensuitwisseling-zorg-en-veiligheidsdomein
/nieuws/17-11-27/nog-6-maanden-voorbereidingstijd-voor-de-avg
/nieuws/17-11-27/kabinetsreactie-tussenrapportage-schakelteam-naar-kamer
/nieuws/17-11-24/subsidienieuws-over-onderwerpen-en-deadlines-eu-programmas
/nieuws/17-11-24/nieuwe-toepassingen-maken-burgernet-nog-effectiever
/nieuws/17-11-24/abp-maakt-pensioenpremie-2018-bekend
/nieuws/17-11-24/workshop-in-de-koerdische-regio-van-irak
/nieuws/17-11-24/staat-van-de-woningmarkt-2017-verschenen
/nieuws/17-11-24/niet-meer-gemeentelijke-uitvaarten
/nieuws/17-11-24/belastingdienst-maakt-notitie-clustering-openbaar-vpb
/nieuws/17-11-24/stevig-ouderschap-ook-in-uw-gemeente
/nieuws/17-11-24/bijeenkomsten-uitwerking-reparatie-flo-overgangsrecht
/nieuws/17-11-24/terugblik-commissie-europa-en-internationaal-november-2017
/nieuws/17-11-24/b

/nieuws/17-11-06/gezamenlijke-omzetting-genormaliseerde-cao-gemeenten
/nieuws/17-11-03/aanpak-van-het-regionaal-expertiseteam-in-midden-brabant
/nieuws/17-11-03/vng-cie-wil-pas-op-de-plaats-verdeelmodel-beschermd-wonen
/nieuws/17-11-03/tot-ziens-bij-de-buitengewone-alv-en-bestuurdersdag
/nieuws/17-11-03/hoe-bereidt-schagen-zich-voor-op-de-omgevingswet
/nieuws/17-11-03/cijfers-jeugdhulp-jeugdbescherming-en-reclassering-in-2017
/nieuws/17-11-03/terugblik-op-oktobervergadering-vng-commissie-gw
/nieuws/17-11-03/evaluatie-gebruik-instrumenten-aanpak-malafide-pandeigenaren
/nieuws/17-11-02/aanpassing-vergoedingen-politieke-ambtsdragers-per-2018
/nieuws/17-11-02/vng-inzet-voor-wijziging-boa-beleidsregels
/nieuws/17-11-02/stappen-tweede-kamer-nodig-voor-versnellen-energietransitie
/nieuws/17-11-02/toolkit-voor-aanpak-oneerlijke-incassopraktijken
/nieuws/17-11-02/uit-de-media-regionale-economie-arbeidsmarkt-oktober
/nieuws/17-11-02/jongeren-tussen-wal-en-schip-toolkit-16-27-helpt-gemeenten
/nie

...

/nieuws/16-07-04/nieuwe-cijfers-huisvesting-vluchtelingen
/nieuws/16-07-04/website-ibewustzijn-overheid-vernieuwd
/nieuws/16-07-04/normbedragen-kwijtschelding-belastingen-per-1-juli-2016
/nieuws/16-07-04/terugblik-commissie-gezondheid-en-welzijn-6-juni
/nieuws/16-07-04/welke-psychosociale-hulpverlening-is-er-voor-vluchtelingen
/nieuws/16-07-04/wrr-overheid-overschat-financiele-zelfredzaamheid-burgers
/nieuws/16-07-04/wrr-adviseert-drie-sporen-voor-schuldhulpverlening
/nieuws/16-07-04/nieuw-portaal-voor-de-budgethouder-pgb
/nieuws/16-07-04/flexibel-cameratoezicht-en-de-inzet-van-drones
/nieuws/16-07-01/verbod-op-alfahulpen-wmo-is-onnodig-en-belemmerend
/nieuws/16-07-01/signaal-van-ouders-geef-concrete-informatie-over-jeugdhulp
/nieuws/16-07-01/terugblik-vng-commissie-jeugd-vergadering-23-juni
/nieuws/16-07-01/wethoudersbijeenkomsten-opvang-en-integratie-vluchtelingen
/nieuws/16-07-01/raad-op-zaterdag-24-september-workshops-bekend
/nieuws/16-07-01/aanleveren-clientervaringsgegevens-wmoje

/nieuws/16-06-14/visie-op-ketensamenwerking-kindermishandeling-en-geweld
/nieuws/16-06-14/terugblik-commissie-ruimte-en-wonen-juni-2016
/nieuws/16-06-14/stemminguitslagen-per-gemeente-alv-8-juni-2016
/nieuws/16-06-14/nieuwe-modelverordening-rechtspositie-politieke-ambtsdragers
/nieuws/16-06-14/schulden-eerder-signaleren-en-schuldeisers-verenigen
/nieuws/16-06-13/uw-experiment-vernieuwing-lokale-democratie-al-aangemeld
/nieuws/16-06-13/voorbeelden-van-integratie-en-participatie-van-vluchtelingen
/nieuws/16-06-13/werkbezoek-armoede-en-schuldenproblematiek-enschede
/nieuws/16-06-13/rekenbestand-integratie-uitkering-jeugd-is-aangepast
/nieuws/16-06-13/coelo-onderzoek-naar-verleggen-kabels-en-leidingen
/nieuws/16-06-13/oproep-tot-deelname-proeftuinen-maak-verschil
/nieuws/16-06-13/pact-moet-aantal-thuiszitters-terugdringen
/nieuws/16-06-12/uw-inbreng-voor-amvbs-omgevingswet-al-aangeleverd
/nieuws/16-06-10/aansluiting-landelijke-voorziening-woz-vraagt-actie
/nieuws/16-06-10/pilots-clienterva

/nieuws/16-05-24/update-ontwikkelingen-omgevingswet
/nieuws/16-05-24/eindelijk-een-breed-moratorium-schuldhulpverlening
/nieuws/16-05-24/uitkomst-bestuurlijk-overleg-over-amvb-inkoop-wmo
/nieuws/16-05-24/tk-bespreekt-visie-rechtspositie-politieke-ambtsdragers-0
/nieuws/16-05-24/raadpleging-uitwerkingsakkoord-asielinstroom-sluit-1-juni
/nieuws/16-05-24/eu-hof-geen-btw-aftrek-leerlingenvervoer-voor-gemeenten
/nieuws/16-05-23/nuttige-standaardartikelen-voor-zorgcontracten-wmo-en-jeugd
/nieuws/16-05-23/terugblik-cie-milieu-energie-en-mobiliteit-april-2016
/nieuws/16-05-23/vng-zet-eerste-stap-in-project-gt-vaste-communicatie
/nieuws/16-05-23/nieuwe-notities-objectvrijstellingen-vennootschapsbelasting
/nieuws/16-05-20/wordversies-vragenlijsten-clientervaring-jeugd-beschikbaar
/nieuws/16-05-20/brabantse-gemeenten-werken-samen-aan-invoering-omgevingswet
/nieuws/16-05-20/terugblik-cie-dienstverlening-en-informatiebeleid-april-2016
/nieuws/16-05-20/digitaal-veiligheidshandboek-verwarde-personen-

/nieuws/16-04-25/bestuurlijk-in-gesprek-over-ambities-afvalscheiding
/nieuws/16-04-25/brandweerkamer-en-bonden-blijven-in-gesprek
/nieuws/16-04-22/uit-de-media-regionale-economie-arbeidsmarkt
/nieuws/16-04-22/uitbreiding-en-wijziging-in-de-wet-normering-topinkomens
/nieuws/16-04-22/zet-maatschappelijke-opgaven-centraal-bij-gebiedsproces
/nieuws/16-04-22/voortgang-toekomstvisie-beschermd-wonen
/nieuws/16-04-22/flo-overgangsrecht-wijziging-leeftijdsafhankelijke-factoren
/nieuws/16-04-22/rijk-koepels-sluiten-akkoord-kosten-invoering-omgevingswet
/nieuws/16-04-22/duurzaamheidstour-ondersteunt-uw-gemeentelijk-sportbeleid
/nieuws/16-04-22/hoe-pakt-de-nieuwe-uitkeringswet-in-de-praktijk-uit
/nieuws/16-04-21/utrecht-uitgeroepen-tot-global-goals-city
/nieuws/16-04-21/integratie-participatie-vluchtelingen-bij-raad-op-zaterdag
/nieuws/16-04-21/rob-advies-met-pleidooi-andere-invulling-rol-raadslid%20
/nieuws/16-04-21/luisteren-naar-het-verhaal-van-een-vluchteling
/nieuws/16-04-21/taaloffensief-teg

ConnectionError: HTTPSConnectionPool(host='vng.nl', port=443): Max retries exceeded with url: /nieuws/16-04-07/onderwijslokaalnl-officieel-van-start (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f290dd6e4a8>: Failed to establish a new connection: [Errno 110] Connection timed out',))

In [5]:
print(len(result))

3109


In [6]:
with open('../data_resources/topics/vng_training.json', 'w') as outfile:
    json.dump(result, outfile)

In [7]:
data = json.loads('../data_resources/topics/vng_training.json')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)